Continue working on the tree after basalv6. 
The aim is to remove rogue taxa and thereby hopefully increase overal support for the tree.

I'll remove:
* all picea abies
* Chlre_v5_5-Cre11_g467577_t1_1-Chlamydomonas_reinhardtii
* Equisetum
* tr_Q5KTX4_Q5KTX4_9VIRI_MADS-box_protein_CpMADS1__Fragment__OS_Closterium_peracerosum-strigosum-littorale_complex_OX_34146_GN_CpMADS1_PE_2_SV_1
* Pp014G086100
* XMGP-2008586-Juniperus_scopulorum

Finally, I will also remove all but one of the algal outgroup sequences, the solidity of this outgroup has been very reproducible.


I'm running trees initially with UFbootstrap and SH-alrt and later with a non-parametric tree inference.

I'm getting the basalv5 fasta and remove everything I don't want.

In [11]:
grep -v -f <(echo -e 'MA_\nChlre\nEquisetum\nQ5KTX4\nPp014G086100\nXMGP-2008586') \
    data/MIKC_orthogroup_selection-basal-v6_guide-v4.fasta \
    > data/MIKC_orthogroup_selection-basal-v7_guide-v4_TEMP.fasta

In [12]:
grep -c '>' data/MIKC_orthogroup_selection-basal-v6_guide-v4.fasta
grep -c '>' data/MIKC_orthogroup_selection-basal-v7_guide-v4_TEMP.fasta

188
171


And finally remove all but one of the algal outgroup sequences:

In [14]:
grep -v -f <(echo -e 'Q5KU24\nQ5KTX4\nMNNM\nQPDY\nFQLP\nTPHT\nTPHT') \
    data/MIKC_orthogroup_selection-basal-v7_guide-v4_TEMP.fasta \
    > data/MIKC_orthogroup_selection-basal-v7_guide-v4.fasta
rm data/MIKC_orthogroup_selection-basal-v7_guide-v4_TEMP.fasta

In [16]:
grep -c '>' data/MIKC_orthogroup_selection-basal-v7_guide-v4.fasta

165


In [17]:
inseq='MIKC_orthogroup_selection-basal-v7_guide-v4'

# aligning and trimming

In [18]:
conda activate phylogenetics
if    [ ! -d ./data/alignments_raw/ ]
then  mkdir  ./data/alignments_raw
fi
if    [ ! -f "./data/alignments_raw/$inseq"_aligned-mafft-linsi.fasta ]
then  einsi --thread $(nproc) data/$inseq.fasta > ./data/alignments_raw/"$inseq"_aligned-mafft-einsi.fasta
fi
conda deactivate

(phylogenetics) (phylogenetics) outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
All-to-all alignment.
tbfast-pair (aa) Version 7.407
alg=N, model=BLOSUM62, 2.00, -0.00, -0.00, noshift, amax=0.0
4 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
  160 / 165
done.

Progressive alignment ... 
STEP    88 /164 (thread    0) 
STEP    90 /164 (thread    1) done. *alloclen = 3899
STEP   164 /164 (thread    3) 
done.
tbfast (aa) Version 7.407
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
4 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 4
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
nadd = 16
Loading 'hat3' ... done.

  160 / 165
Segment   1/  1    1-3049
005-0326-0 (thread    3) worse         
Converged2.
done
dvtditr (aa) Version 7.407
alg=A, model=BLOSUM62, 1.53

In [20]:
conda activate phylogenetics
if    [ ! -d data/alignments_trimmed ]
then  mkdir  data/alignments_trimmed 
fi

gt=( 1 2 3 4 5 )

# define appendix only once here:
for g in ${gt[@]}
do  trimappendix=trim-gt"$g"
    for a in "data/alignments_raw/$inseq"_aligned*.fasta
    do  appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
        if    [ ! -f "$appendix"_"$trimappendix".fasta ]
        then  echo "trimming alignment $a with gap threshold $g"
              sed -i 's/ /_/g' $a
              trimal -in $a   \
                     -out data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".fasta \
                     -gt ."$g" \
                     -htmlout data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".html &
        fi
    done
done
wait
conda deactivate

(phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) trimming alignment data/alignments_raw/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi.fasta with gap threshold 1
[1] 486823
trimming alignment data/alignments_raw/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi.fasta with gap threshold 2
[2] 486831
trimming alignment data/alignments_raw/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi.fasta with gap threshold 3
[3] 486838
trimming alignment data/alignments_raw/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi.fasta with gap threshold 4
[4] 486845
trimming alignment data/alignments_raw/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi.fasta with gap threshold 5
[5] 486852
(phylogenetics) [1]   Done                    trimal -in $a -out data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".fasta -gt ."$g" -htmlout data/alignments_trimmed/"$inseq"_"$appendix"_"$t

In [ ]:
conda activate jalview
for   i in data/alignments_trimmed/*.fasta
do    prefix=$(echo $i | sed 's/\.fasta//')
      if    [ ! -f $prefix.png ]
      then  jalview -nodisplay \
                    -open $prefix.fasta \
                    -colour CLUSTAL \
                    -png  $prefix.png > /dev/null 2> /dev/null &
      fi
done
wait
conda deactivate

(jalview) [1] 487881
[2] 487888
[3] 487898
[4] 487915
[5] 487922
[6] 487926
[7] 487958
[8] 487972
[9] 487981
[10] 487993
[11] 488004
[12] 488044
[13] 488056
[14] 488078
[15] 488087
[16] 488100
[17] 488140
[18] 488165
[19] 488191
[20] 488200
[21] 488212
[22] 488229
[23] 488246
[24] 488259
[25] 488268
[26] 488288
[27] 488298
[28] 488307
[29] 488317
(jalview) 

The Final alignment looks like this, quite happy with it. 

![image.png]()


# tree inference
Now let's get to tree inference, choosing a fast bootstrapping method and reusing a model from a previous fit of a practically similar alignment.

In [1]:
inseq=MIKC_orthogroup_selection-basal-v6_guide-v4
ls data/alignments_trimmed/"$inseq"_aligned*fasta

data/alignments_trimmed/MIKC_orthogroup_selection-basal-v6_guide-v4_aligned-mafft-einsi_trim-gt1.fasta
data/alignments_trimmed/MIKC_orthogroup_selection-basal-v6_guide-v4_aligned-mafft-einsi_trim-gt1-seqrmmanual.fasta


In [2]:
conda activate phylogenetics

(phylogenetics) 

: 1

In [3]:
a=data/alignments_trimmed/MIKC_orthogroup_selection-basal-v6_guide-v4_aligned-mafft-einsi_trim-gt1-seqrmmanual.fasta

#iqpendix='iqtree-b100'
iqpendix='iqtree-LGR6-bb2000-alrt2000'

echo "making a tree of file $a"
echo "The first lines of alignment $a look like this"
head $a

file_appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
echo "Making a directory $file_appendix to store trees (name based on alignment filename)"

if   [ ! -d    analyses/"$inseq"_trees/"$file_appendix" ]
then mkdir -p  analyses/"$inseq"_trees/"$file_appendix" 
fi

iqprefix=analyses/"$inseq"_trees/"$file_appendix"/"$inseq"_"$file_appendix"_"$iqpendix"
if   [ ! -f "$iqprefix".treefile ]
then nice iqtree -s $a \
                 -m 'LG+R6' \
                 -bb 2000 -alrt 2000 \
                 -nt 4 \
                 -pre  "$iqprefix" \
                 2>   "$iqprefix".stderr \
                 >    "$iqprefix".stdout && cat "$iqprefix".log | mail -s IQtree_run laura
fi

(phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) making a tree of file data/alignments_trimmed/MIKC_orthogroup_selection-basal-v6_guide-v4_aligned-mafft-einsi_trim-gt1-seqrmmanual.fasta
(phylogenetics) The first lines of alignment data/alignments_trimmed/MIKC_orthogroup_selection-basal-v6_guide-v4_aligned-mafft-einsi_trim-gt1-seqrmmanual.fasta look like this
(phylogenetics) >tr|Q5KU24|Q5KU24_COLSC_MADS-box_transcription_factor_CsMADS1_OS=Coleochaete_scutata_OX=3125_GN=csmads1_PE=2_SV=1
---------------------------------------------MGRGKIEIRRIENAT
SRQVTFSKRRNGLLKKAYELSVLCDVDIAVIVFSPTGKLFQYASS-SMKEILERYEQVPP
EQ--KEKGSQRLDNMDYLNREVAKLRNEVEHKYHEARQLEGEDLDRLGVYELEQLEQKLS
NSMRRIRGRKDELMKAELEGLRKQVADLSGSSNYLLQSIPGIRTMPPSSLGGMNPASTSL
ASDESPVSVDFAQAPREMSGVDLS------GSPVPPWKSQAAAAAQQTDWKVTNTEHLDS
WPKAPAP--STSVQPEWKN-QSSPSSE-------WMY
>tr|Q5KTX4|Q5KTX4_9VIRI_MADS-box_protein_CpMADS1_(Fragment)_OS=Closterium_peracerosum-strigosum-littorale_complex_OX=34146_GN=CpMADS1_P

: 1

In [ ]:
tail -n 30 $iqprefix.log

In [12]:
grep 'Best-fit model' $iqprefix.log

Best-fit model: JTT+R8 chosen according to BIC
(phylogenetics) 

: 1

In [ ]:
a=data/alignments_trimmed/MIKC_orthogroup_selection-basal-v6_guide-v4_aligned-mafft-einsi_trim-gt1-seqrmmanual.fasta

iqpendix='iqtree-MFP-b1000'
#iqpendix='iqtree-MFP-bb2000-alrt2000'

echo "making a tree of file $a"
echo "The first lines of alignment $a look like this"
head $a

file_appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
echo "Making a directory $file_appendix to store trees (name based on alignment filename)"

if   [ ! -d    analyses/"$inseq"_trees/"$file_appendix" ]
then mkdir -p  analyses/"$inseq"_trees/"$file_appendix" 
fi

iqprefix=analyses/"$inseq"_trees/"$file_appendix"/"$inseq"_"$file_appendix"_"$iqpendix"
if   [ ! -f analyses/"$inseq"_trees/"$file_appendix"/"$inseq"_"$file_appendix"_iqtree-bb1000.tree ]
then nice iqtree -s $a    \
                 -m JTT+R8  \
                 -b 1000  \
                 -nt 4 \
                 -pre  "$iqprefix" \
                 2>   "$iqprefix".stderr \
                 >    "$iqprefix".stdout && cat "$iqprefix".log | mail -s IQtree_run laura
fi

(phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) making a tree of file data/alignments_trimmed/MIKC_orthogroup_selection-basal-v6_guide-v4_aligned-mafft-einsi_trim-gt1-seqrmmanual.fasta
(phylogenetics) The first lines of alignment data/alignments_trimmed/MIKC_orthogroup_selection-basal-v6_guide-v4_aligned-mafft-einsi_trim-gt1-seqrmmanual.fasta look like this
(phylogenetics) >tr|Q5KU24|Q5KU24_COLSC_MADS-box_transcription_factor_CsMADS1_OS=Coleochaete_scutata_OX=3125_GN=csmads1_PE=2_SV=1
---------------------------------------------MGRGKIEIRRIENAT
SRQVTFSKRRNGLLKKAYELSVLCDVDIAVIVFSPTGKLFQYASS-SMKEILERYEQVPP
EQ--KEKGSQRLDNMDYLNREVAKLRNEVEHKYHEARQLEGEDLDRLGVYELEQLEQKLS
NSMRRIRGRKDELMKAELEGLRKQVADLSGSSNYLLQSIPGIRTMPPSSLGGMNPASTSL
ASDESPVSVDFAQAPREMSGVDLS------GSPVPPWKSQAAAAAQQTDWKVTNTEHLDS
WPKAPAP--STSVQPEWKN-QSSPSSE-------WMY
>tr|Q5KTX4|Q5KTX4_9VIRI_MADS-box_protein_CpMADS1_(Fragment)_OS=Closterium_peracerosum-strigosum-littorale_complex_OX=34146_GN=CpMADS1_P

The non parametric tree shows bootstrap 0 for a lot of ancestral nodes in the tree, and again it is clear some taxa move around a lot.
I have a theory that these rogue taxa are detrimental to the overal support of the tree and want to carefully remove some.
Hence, I identified these by a combination of
 1. comparing different trees, predominantly the UFbootstrap and the normal bootstrap tree of v6, to see which taxa move around a lot
 2. finding exceptionally long branches or oddly placed branches, like the algal taxa.
 3. Again carefully inspecting the MSA.
 
This has led me to the conclusion I'll remove:
* all picea abies
* Chlre_v5_5-Cre11_g467577_t1_1-Chlamydomonas_reinhardtii
* Equisetum
* tr_Q5KTX4_Q5KTX4_9VIRI_MADS-box_protein_CpMADS1__Fragment__OS_Closterium_peracerosum-strigosum-littorale_complex_OX_34146_GN_CpMADS1_PE_2_SV_1
* Pp014G086100
* XMGP-2008586-Juniperus_scopulorum

Finally, I will also remove all but one of the algal outgroup sequences, the solidity of this outgroup has been very reproducible.
I was inclined to remove the Selaginella moelendorfii sequences, but these get the benefit of the doubt. Also, by the criteria mentioned here, Arabidopsis FLC should also be removed, but I chose to keep because... well it's arabidopsis FLC.

A snapshot of this tree:
![basalv6 tree](analyses/MIKC_orthogroup_selection-basal-v6_guide-v4_trees/aligned-mafft-einsi_trim-gt1-seqrmmanual/MIKC_orthogroup_selection-basal-v6_guide-v4_aligned-mafft-einsi_trim-gt1-seqrmmanual_iqtree-MFP-b1000.png)
And the link to itol: https://itol.embl.de/tree/942102157910201593716454#